In [22]:
!pip install --upgrade pip
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension install --py widgetsnbextension
!pip install jupyterlab_widgets
!pip install transformers
!pip install torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


import torch
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
text = "Replace me by any text you'd like."
model = model.to(device)
encoded_input = tokenizer(text, return_tensors='pt').to(device)
output = model(**encoded_input)

# Load the transformer model and tokenizer
#model_name = "https://huggingface.co/FacebookAI/roberta-base"  
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModel.from_pretrained(model_name)
#model = model.to(device)  # Move to device (GPU or CPU)

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [

ImportError: 
RobertaModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [18]:
# Load the dataset from the provided gzip file
import gzip

# Path to the dataset
dataset_path = "assignment4-dataset.txt.gz"

# Read dataset into a list of sentences
dataset_sentences = []
with gzip.open(dataset_path, 'rt') as file:
    dataset_sentences = [line.strip() for line in file if line.strip()]  # Exclude empty lines

print(f"Loaded dataset with {len(dataset_sentences)} sentences.")

Loaded dataset with 3980290 sentences.


In [19]:
# Tokenize a subset of the dataset for demonstration purposes
subset_sentences = dataset_sentences[:100]  # Process the first 100 sentences as a demonstration
tokenized_inputs = tokenizer(subset_sentences, padding=True, truncation=True, return_tensors="pt")
tokenized_inputs = {key: val.to(device) for key, val in tokenized_inputs.items()}  # Move tensors to the correct device
print("Tokenization complete for 100 sentences.")


ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.

In [15]:
# Pass tokenized inputs through the model
with torch.no_grad():
    outputs = model(**tokenized_inputs)
contextual_embeddings = outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)
    

NameError: name 'model' is not defined

In [16]:
# Compute static embeddings by averaging contextualized embeddings for each token
token_embeddings = defaultdict(list)

for i, text in enumerate(texts):
    tokens = tokenizer.tokenize(text)
    for j, token in enumerate(tokens):
        token_embeddings[token].append(contextual_embeddings[i, j].cpu().numpy())  # Move to CPU for numpy

# Compute average embedding for each token
static_embeddings = {token: np.mean(embeds, axis=0) for token, embeds in token_embeddings.items()}

# Save the embeddings for later use
import pickle
with open("static_embeddings.pkl", "wb") as f:
    pickle.dump(static_embeddings, f)

print(f"Generated static embeddings for {len(static_embeddings)} unique tokens.")

NameError: name 'texts' is not defined